In [29]:
import json
import pandas as pd
import gurobipy as gp
from datetime import datetime, timedelta

In [2]:
OBP_AVG = 0.327
SLG_AVG = 0.365

FPCT_AVG = {
    '2': 0.992,
    '3': 0.993,
    '4': 0.977, 
    '5': 0.929,
    '6': 0.964,
    '7': 0.977,
    '8': 0.988,
    '9': 0.976
}

POSITION_WEIGHT = {
    '2': {"batting": 0.95, "fielding": 1.05},
    '3': {"batting": 1.21, "fielding": 0.83},
    '4': {"batting": 0.97, "fielding": 1.03},
    '5': {"batting": 0.96, "fielding": 1.04},
    '6': {"batting": 0.94, "fielding": 1.06},
    '7': {"batting": 1.02, "fielding": 0.98},
    '8': {"batting": 1.04, "fielding": 0.96},
    '9': {"batting": 1.17, "fielding": 0.85},
}

OPPONENT_WEIGHT = {
    "brothers": 1.15,
    "lions": 1.11,
    "monkeys": 0.96, 
    "guardians": 0.93,
    "dragons": 0.85
}

dataRoot = "../clean/clean_all_"
playerRoot = "../球員異動/"

teamNameMap = {
    "中信兄弟": "brothers", 
    "味全龍": "dragons",
    "富邦悍將": "guardians", 
    "統一7-ELEVEn獅": "lions",
    "樂天桃猿": "monkeys"
}

### Import Data

In [3]:
class Player:
    def __init__(self, name, data):
        self.name = name
        self.data = data

        # A_{ij}
        canPlay = []
        for i in range(9):
            if (i + 1) in data["fielding"]["pos"]:
                canPlay.append(1)
            else:
                canPlay.append(0)
        self.canPlay = canPlay

        # F_{ij}
        F = {}
        for i in data["fielding"]["pos"]:
            F[str(i)] = float(data["fielding"]["FPCT"][str(i)]) / FPCT_AVG[str(i)]
            if F[str(i)] == 1:
                F[str(i)] = 0.98
            if F[str(i)] < 0.8:
                F[str(i)] = 0.8
        self.F = F

In [4]:
allData = {}
for team in teamNameMap.keys():
    with open(dataRoot + team + ".json", encoding="utf-8") as f:
        temp = []
        for line in f:
            player = json.loads(line)
            temp.append(Player(player["name"], player["data"]))
        allData[teamNameMap[team]] = temp

brothers = allData["brothers"]
dragons = allData["dragons"]
guardians = allData["guardians"]
lions = allData["lions"]
monkeys = allData["monkeys"]

In [21]:
playerList = {}

for team in teamNameMap.keys():
    tempDict = pd.read_csv(playerRoot + team + ".csv", header=0, index_col=0).to_dict()
    tempDict = {
        datetime.strptime(key, "%Y-%m-%d").date(): list(filter(lambda d: pd.isna(d) == False, list(value.values())))
        for key, value in tempDict.items()
    }
    playerList[teamNameMap[team]] = tempDict

In [6]:
def createDate(mm, dd):
    return datetime(2022, mm, dd, 0, 0, 0).date()

### Self-defined Parameters

In [22]:
aTeam = "中信兄弟"
aMonth = "May"
aDate = [createDate(5, 3), createDate(5, 5), createDate(5, 6), createDate(5, 7), createDate(5, 8)]
aField = ["新莊", "澄清湖", "洲際", "洲際", "洲際"]
aOppo = ["富邦悍將", "味全龍", "統一7-ELEVEn獅", "統一7-ELEVEn獅", "統一7-ELEVEn獅"]
aPitcher = ["陳鴻文", "王維中", "江辰晏", "李其峰", "邱浩鈞"]
aStar = ["許基宏", "江坤宇"]
aPotential = ["岳東華"]
aInjured = ["周思齊"]

### Optimization

In [35]:
# 比賽當天 27 人名單
def findPlayerList(team, date):
    minDelta = timedelta(days=100)
    playerDay = date
    for key in playerList[teamNameMap[aTeam]].keys():
        if date > key:
            break
        if key - date < minDelta:
            minDelta = key - date
            playerDay = key
    print(playerDay, playerList[teamNameMap[team]][playerDay])
        

In [37]:
findPlayerList("中信兄弟", createDate(5, 17))

2022-05-17 ['廖乙忠', '鄭凱文', '泰迪', '李振昌', '德保拉', '象魔力', '楊志龍', '呂彥青', '陳柏豪', '吳哲源', '陳家駒', '高宇杰', '黃鈞聲', '王威晨', '王政順', '杜家明', '許基宏', '江坤宇', '岳東華', '陳子豪', '李聖裕', '陳文杰', '張志豪', '周思齊', '詹子賢', '岳政華', '吳俊偉']


In [ ]:
model = gp.Model("model")

# x_{ijg}: player i play position j in game g
